In [1]:
## Task: 
## (1) Which keywords can let your comments get a higher score in Reddit? 
## 2. Compared the run time of (1) in common way and pyspark


import pandas as pd

with open('data/RC_2006-01') as f:
    data = pd.read_json(f, lines=True)

In [2]:
column_names = data.columns.tolist()
print(column_names)

['subreddit', 'author_flair_css_class', 'created_utc', 'score', 'ups', 'body', 'controversiality', 'link_id', 'stickied', 'subreddit_id', 'gilded', 'retrieved_on', 'distinguished', 'author_flair_text', 'author', 'parent_id', 'edited', 'id']


In [3]:
# pre processing
data = data[data['body'].str.len() >= 10]
data = data.drop(data[data['body'] == '[deleted]'].index)

In [4]:
data = data.loc[:, ['score', 'body']]
data.info

<bound method DataFrame.info of       score                                               body
0         0                         early 2006 a probable date
1         0  If you are going to post something that has a ...
2         2    Microsoft hates it's own products?\r\nWho knew?
3         2  this looks interesting, but it's already aired...
4         0  I have nothing but good things to say about De...
...     ...                                                ...
3661      4  I find his "I am a mathematician" approach kin...
3662     -5  I totally did not pick up on that distinction....
3663      4  So, having tact filters on both input and outp...
3664     -7  Err... I think HPK you are closer to being hom...
3665     -1  Here's an example of the lack of diversity of ...

[3067 rows x 2 columns]>

In [5]:
stop_words = ['a', 'an', 'and', 'are', 'as', 'at', 'be', 'but', 'by', 'for', 'if', 'in', 'into', 'is', 'it',
              'no', 'not', 'of', 'on', 'or', 'such', 'that', 'the', 'their', 'then', 'there', 'these', 'they',
              'this', 'to', 'was', 'will', 'with', '!', '@', '#', '$', '%', '^', '&', '*', '(', ')', '-', 
              '_', '+', '=', '[', ']', '{', '}', ';', ':', '"', "'", '<', '>', ',', '.', '/', '?', '\\', 
              '|', '`', '~', '']

In [6]:
word_score = {}
word_count = {}
for index, row in data.iterrows():
    score = row['score']
    words_list = row['body'].split()
    for word in words_list:
        # filter stop word
        if word not in stop_words:
            if word not in word_count.keys():
                word_count[word] = 1
                word_score[word] = score
            else:
                word_count[word] = 1 + word_count[word]
                word_score[word] = score + word_score[word]

In [7]:
filtered_word_count = {k: v for k, v in word_count.items() if v >= 5}  # filter word appear less than 10 times
filtered_word_score = {k: v for k, v in word_score.items() if k in filtered_word_count.keys()}
print(len(word_count.keys()))
print(len(word_score.keys()))
print(len(filtered_word_score.keys()))
print(len(filtered_word_count.keys()))

24191
24191
2897
2897


In [8]:
for word,times in filtered_word_count.items():
    if filtered_word_score[word] != 0:
        filtered_word_score[word] = filtered_word_score[word]/times
    else:
        pass

In [9]:
sorted_filtered_word_score = dict(sorted(filtered_word_score.items(), key=lambda x: x[1], reverse=True))

# top 10 word  that may let your comments get a higher score in Reddit
for key, value in list(sorted_filtered_word_score.items())[:10]:
    print(f"{key}: {value}")

Viaweb: 19.5
books.: 11.5
money,: 10.5
spoken: 9.6
Stossel: 9.4
about,: 9.2
loved: 8.833333333333334
efficient: 8.8
You've: 8.6
technical: 8.5
